In [1]:
from decimal import Decimal, getcontext
from fractions import Fraction
import numpy as np

Часто встречаются ситуации, когда, казалось бы, элементарные операции с числами с плавающей точкой дают неожиданные результаты. Классический пример: вычисление суммы. Простая сумма двух чисел с плавающей точкой может дать неверный результат. При увеличении количества чисел операция суммирования становится еще и некоммутативной.
См., например, https://habr.com/ru/post/526000/

In [2]:
0.1 + 0.2 == 0.3

False

Для сверхточных вычислений простых числовых функций в python есть два модуля стандартной библиотеки: 
1. `decimal`, где реализованы числа с фиксированной точкой и произвольной точностью.
2. `fractions`, где реализованы рациональные числа

### Числа с фиксированной точкой
Числа типа `Decimal` могут быть инициализированы с произвольной точностью из `float` или строки и лишены недостатков чисел с плавающей точкой, связанных с двоичным представлением. Если инициализировать `Decimal` из строки, то количество знаков фиксируется вводом:

In [3]:
Decimal('0.1') + Decimal('0.2')

Decimal('0.3')

Иначе могут быть не совсем корректные результаты, так как идет преобразование из типа `float`, которые могут быть определены неточно:

In [4]:
Decimal(0.1) + Decimal(0.2)

Decimal('0.3000000000000000166533453694')

Для управления точностью вычислений есть функция `decimal.getcontext()`

In [5]:
getcontext().prec = 8 # Точность в десятичных знаках после запятой
getcontext()

Context(prec=8, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])

И вот уже ненужные знаки после запятой отсечены

In [6]:
Decimal(0.1) + Decimal(0.2)

Decimal('0.30000000')

### Рациональные числа
Рациональные числа, то есть числа, выражаемые через отношение целого и натурального чисел, полезны при хранении дробных чисел с конечной точностью и выполнения с ними различных арифметических операций без потери точности. Может быть задана через строку, `float` или `Decimal`

In [7]:
Fraction(Decimal('0.5')), Fraction('0.5'), Fraction(0.5)

(Fraction(1, 2), Fraction(1, 2), Fraction(1, 2))

### Интеграция с `numpy`
Для работы с числами удобно пользоваться массивами `numpy`, в которые с определенными ограничениями можно интегрировать и `Fraction`, и `Decimal`. Сначала разберем, что сделать не получится. Не получится воспользоваться аргументом `dtype` и прямым преобразованием типов:

In [8]:
type(np.ones(10, dtype=Decimal)[0]) == Decimal

False

In [9]:
type(np.ones(10).astype(Fraction)[0]) == Fraction

False

Есть универсальный метод создания массивов с нужным типом на основе поэлементного применения соответствующих преобразований типов:

In [10]:
test_array1 = np.asarray([Decimal(x) for x in np.ones(10)])
test_array1

array([Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'),
       Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'),
       Decimal('1'), Decimal('1')], dtype=object)

То же самое можно сделать, создав векторизованную функцию с помощью декоратора `np.vectorize`:

In [11]:
@np.vectorize
def float2decimal(x):
    return Decimal(x)

test_array2 = float2decimal(np.ones(10))
test_array2

array([Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'),
       Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'),
       Decimal('1'), Decimal('1')], dtype=object)

Для рациональных чисел есть еще один трюк, работающий с массивами в целых числах, основанный на том факте, что определен результат сложения целого и рационального чисел:

In [12]:
np.ones(10, dtype=int) + Fraction()

array([Fraction(1, 1), Fraction(1, 1), Fraction(1, 1), Fraction(1, 1),
       Fraction(1, 1), Fraction(1, 1), Fraction(1, 1), Fraction(1, 1),
       Fraction(1, 1), Fraction(1, 1)], dtype=object)

### Математические операции с `Decimal` и `Fraction`
Для этих типов чисел отсутствуют многие стандартные, например, тригонометрические, функции. Функции `numpy` не работают вовсе, функции из стандартной библиотеки `math` возвращают `float`. Также понятно, что тригонометрические и многие другие иррациональные функции не могут быть реализованы для `Fraction`.
Полный набор реализованных функций можно посмотреть в официальной документации:
1. https://docs.python.org/3/library/decimal.html
2. https://docs.python.org/3/library/fractions.html

In [13]:
import math
test_array3 = np.vectorize(math.sin)(test_array2)
test_array3, test_array3.dtype

(array([0.84147098, 0.84147098, 0.84147098, 0.84147098, 0.84147098,
        0.84147098, 0.84147098, 0.84147098, 0.84147098, 0.84147098]),
 dtype('float64'))

Для реализации тех же тригонометрических функций для `Decimal` даже [официальная документация](https://docs.python.org/3/library/decimal.html#recipes) предлагает использовать самописные функции, например такие:

In [14]:
def cos(x):
    """Return the cosine of x as measured in radians.

    The Taylor series approximation works best for a small value of x.
    For larger values, first compute x = x % (2 * pi).

    >>> print(cos(Decimal('0.5')))
    0.8775825618903727161162815826
    >>> print(cos(0.5))
    0.87758256189
    >>> print(cos(0.5+0j))
    (0.87758256189+0j)

    """
    getcontext().prec += 2
    i, lasts, s, fact, num, sign = 0, 0, 1, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 2
        fact *= i * (i-1)
        num *= x * x
        sign *= -1
        s += num / fact * sign
    getcontext().prec -= 2
    return +s

Такие функции основаны на цикле `while` и работают гораздо медленнее аналогов для `float` из стандартной библиотеки или `numpy`

In [15]:
a = Decimal('0.5')
%timeit cos(a)

5.77 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%timeit np.sin(0.5)

776 ns ± 7.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [17]:
%timeit math.sin(0.5)

140 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Почему версия `numpy` медленнее? Потому что она оптимизирована для применения с массивами. Пример:

In [18]:
%timeit [math.sin(x) for x in np.ones(1000)]

223 µs ± 4.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%timeit np.sin(np.ones(1000))

11.9 µs ± 85.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
